In [ ]:
import os
import json
import numpy as np
import re
from difflib import SequenceMatcher
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import plotly.graph_objects as go

In [ ]:
# base_folder = "/content/drive/My Drive/SNLP/Response_eval"
base_folder = "/content/drive/My Drive/SNLP/Response_eval"

experiment_folders = [f for f in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, f))]

for experiment_folder in experiment_folders:
    experiment_path = os.path.join(base_folder, experiment_folder)
    json_files = [f for f in os.listdir(experiment_path) if f.endswith(".json")]

    print(f"Checking folder: {experiment_path}")  # Debug print
    print(f"Found JSON files: {json_files}")  # Debug print

    colors = {
        "baseline": "blue",
        "CoT": "green",
        "manually_written": "red"
    }

    models = {
        "baseline": "solid",
        "CoT": "dash",
        "manually_written": "dot"
    }

    k_values = ["k1", "k3", "k5", "k10"]
    metrics = ["bert_f1", "rouge_recall", "faithfulness", "answer_relevancy"]

    for json_file in json_files:
        json_file_path = os.path.join(experiment_path, json_file)
        print(f"Processing JSON file: {json_file_path}")  # Debug print

        with open(json_file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        dataset_name = json_file.replace("_gpt_evaluation_results_reorganized_by_type.json", "")

        # **🔹 1. Plot each metric separately for this dataset**
        for metric in metrics:
            fig = go.Figure()

            for model, line_style in models.items():
                y_values = []
                for k in k_values:
                    # Handling case where metric is inside "ragas_metrics" (stringified dictionary)
                    if metric in data[model][k]:
                        y_values.append(data[model][k][metric])
                    else:
                        y_values.append(eval(data[model][k]["ragas_metrics"])[metric])

                fig.add_trace(go.Scatter(
                    x=k_values, y=y_values,
                    mode="lines+markers",
                    line=dict(dash=line_style),
                    name=f"{model}"
                ))

            # Update layout
            fig.update_layout(
                title=f"{dataset_name} - {metric.capitalize()} vs K",
                xaxis_title="K value",
                yaxis_title=metric.capitalize(),
                hovermode="x unified"
            )

            fig.show()  # Show each plot separately